# **TCLab Closed-Loop PID with FeedForward**

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from IPython.display import display, clear_output
import time
import threading

import package_LAB
from importlib import reload
package_LAB = reload(package_LAB)

from package_LAB import LL_RT, PID_RT, IMCTuning, install_and_import
from package_DBR import SelectPath_RT, Delay_RT, FO_RT

In [98]:
#plotly imports

install_and_import('plotly')
install_and_import('plotly')
install_and_import('ipywidgets')
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from ipywidgets import interactive, VBox, IntRangeSlider, IntSlider, Checkbox, interactive_output
import ipywidgets as widgets


## TCLab parameters

In [99]:
#DV 
Kp_ODV_SOPDT = 0.2951290424136788
T1_ODV_SOPDT = 182.2549613489765
T2_ODV_SOPDT = 13.184430234847984
theta_ODV_SOPDT = 28.999891911961512

#MV
Kp_OMV_SOPDT = 0.30788564834253684
T1_OMV_SOPDT = 183.81942938046797
T2_OMV_SOPDT = 3.2920224028341535e-12
theta_OMV_SOPDT = 20.015407110302775

#Operating points 
DV0 = 50 
MV0 = 50
PV0 = 49.3

# Set maximum and minimum MV values
MVmin = 0
MVmax = 100

# Coefficients
alpha = 0.7
gamma = 0.5

#IMC Tuning
Kc, TI, TD = IMCTuning(Kp_OMV_SOPDT, T1_OMV_SOPDT, T2_OMV_SOPDT, theta_OMV_SOPDT, gamma, model="SOPDT")
print(f"Kc: {Kc}, TI: {TI}, TD: {TD}")

Kc: 5.33426261068635, TI: 183.81942938047126, TD: 3.2920224028340946e-12


## Arrays, Paths and Variables Initialization

In [100]:
#default scenario
SPPath = {0: PV0}
ManPath = {0: False}
MVManPath = {0: MV0}
DVPath = {0: DV0}
FF = True
ManFF = False # Not needed

#Time parameters
TSim = 3000
Ts = 1    
N = int(TSim/Ts) + 1  
TimeStep = Ts # Time step for the simulation


## PID and plot

In [101]:
fig = go.FigureWidget(make_subplots(rows=4, cols=1, specs = [[{}], [{}], [{}], [{}]], vertical_spacing = 0.15, row_heights=[0.1, 0.4, 0.4, 0.1], subplot_titles=("Manual Mode", "MV and Components", "PV, SP and E", "Perturbation DV")))
def initialize():
	fig.add_trace(go.Scatter(name="SP"), row=3, col=1)
	fig.add_trace(go.Scatter(name="PV"), row=3, col=1)
	fig.add_trace(go.Scatter(name="E", line=dict(dash='dash')), row=3, col=1)
	fig.add_trace(go.Scatter(name="MV"), row=2, col=1)
	fig.add_trace(go.Scatter(name="MVp", line=dict(dash='dash')), row=2, col=1)
	fig.add_trace(go.Scatter(name="MVi", line=dict(dash='dash')), row=2, col=1)
	fig.add_trace(go.Scatter(name="MVd", line=dict(dash='dash')), row=2, col=1)
	fig.add_trace(go.Scatter(name="Man"), row=1, col=1)
	fig.add_trace(go.Scatter(name="MVMan"), row=1, col=1)
	fig.add_trace(go.Scatter(name="DV"), row=4, col=1)
	# Update layout
	fig['layout'].update(height=800, width=800)
	fig['layout']['xaxis1'].update(title='Time (s)')
	fig['layout']['yaxis1'].update(title='(°C)')
	fig['layout']['xaxis2'].update(title='Time (s)')
	fig['layout']['yaxis2'].update(title='MV (%)')
	fig['layout']['xaxis3'].update(title='Time (s)')
	fig['layout']['xaxis4'].update(title='Time (s)')
 
def reinitialize():    
	global t, SP, MAN, MV_MAN, DV, PV, MVFF, MV, MVp, MVi, MVd, E, PV_p, PV_d, MVFF_Delay, MVFF_LL1, MV_Delay_P, MV_FO_P, MV_Delay_D, MV_FO_D

	t = []
	SP = []
	PV = []
	MAN = []
	MV_MAN = []
	DV = []
	MVFF = []
	MV = []
	MVp = []
	MVi = []
	MVd = []
	E = []
	PV_p = []
	PV_d = []

	MVFF_Delay = []
	MVFF_LL1 = []
	MV_Delay_P = []
	MV_FO_P = []
	MV_Delay_D = []
	MV_FO_D = []
 
initialize()


In [102]:


def RunExp(Exp):
    global should_continue
    # Reset the flag to True every time RunExp is called
    
    global KcBuffer, TIBuffer, TDBuffer, alphaBuffer, ManPathBuffer, MVmanPathBuffer, FFBuffer, SPPathBuffer, DVPathBuffer
    
    Kc = KcBuffer
    TI = TIBuffer
    TD = TDBuffer
    alpha = alphaBuffer
    ManPath = ManPathBuffer
    MVManPath = MVmanPathBuffer
    FF = FFBuffer
    SPPath = SPPathBuffer
    DVPath = DVPathBuffer
    
    if Exp:
        reinitialize()
        for i in range(0, TSim):
            if not should_continue:
                break  # Exit the loop if should_continue is False
            t.append(i * Ts)
            
            if t[-1] == 0:
                last_time = time.time()
                
            SelectPath_RT(SPPath, t, SP)
            SelectPath_RT(ManPath, t, MAN)
            SelectPath_RT(MVManPath, t, MV_MAN)
            SelectPath_RT(DVPath, t, DV)
            
            # FeedForward
            Delay_RT(DV - DV0*np.ones_like(DV), max(theta_ODV_SOPDT-theta_OMV_SOPDT, 0), Ts, MVFF_Delay)
            LL_RT(MVFF_Delay, -Kp_ODV_SOPDT/Kp_OMV_SOPDT, T1_OMV_SOPDT, T1_ODV_SOPDT, Ts, MVFF_LL1)
            if FF:
                LL_RT(MVFF_LL1, 1, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF)
            else:
                LL_RT(MVFF_LL1, 0, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF) # Set MVFF to 0 if FF is disabled
            
            # PID
            PID_RT(SP, PV, MAN, MV_MAN, MVFF, Kc, TI, TD, alpha, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E, ManFF, PV0)
            
            # Process
            Delay_RT(MV, theta_OMV_SOPDT, Ts, MV_Delay_P, MV0)
            FO_RT(MV_Delay_P, Kp_OMV_SOPDT, T1_OMV_SOPDT, Ts, MV_FO_P)
            FO_RT(MV_FO_P, 1, T2_OMV_SOPDT, Ts, PV_p)
            
            # Disturbance
            Delay_RT(DV - DV0*np.ones_like(DV), theta_ODV_SOPDT, Ts, MV_Delay_D)
            FO_RT(MV_Delay_D, Kp_ODV_SOPDT, T1_ODV_SOPDT, Ts, MV_FO_D)
            FO_RT(MV_FO_D, 1, T2_ODV_SOPDT, Ts, PV_d)
            
            PV.append(PV_p[-1] + PV_d[-1] + PV0 - Kp_OMV_SOPDT*MV0)
            
            if TimeStep == 0:
                last_time = time.time()
            else : 
                # wait to the next loop
                elapsed = time.time() - last_time
                time.sleep(max(0, TimeStep - elapsed))
                last_time = time.time()
            
            # Use extend_traces method to update the plot
            new_data = {
                'x': [[t[-1]]*10],
                'y': [
                    [SP[-1]], [PV[-1]], [E[-1]], [MV[-1]], 
                    [MVp[-1]], [MVi[-1]], [MVd[-1]], 
                    [MAN[-1]], [MV_MAN[-1]], [DV[-1]]
                ]
            }
            fig.extend_traces(new_data, indices=list(range(10)))
            # with fig.batch_update():
            #     fig.data[0].x, fig.data[0].y = t, SP
            #     fig.data[1].x = t
            #     fig.data[1].y = PV
            #     fig.data[2].x = t
            #     fig.data[2].y = E
            #     fig.data[3].x = t
            #     fig.data[3].y = MV
            #     fig.data[4].x = t
            #     fig.data[4].y = MVp
            #     fig.data[5].x = t
            #     fig.data[5].y = MVi
            #     fig.data[6].x = t
            #     fig.data[6].y = MVd
            #     fig.data[7].x = t
            #     fig.data[7].y = MAN
            #     fig.data[8].x = t
            #     fig.data[8].y = MV_MAN
            #     fig.data[9].x = t
            #     fig.data[9].y = DV
                
def Update_gamma(gamma):
    global KcBuffer, TIBuffer, TDBuffer
    KcBuffer, TIBuffer, TDBuffer = IMCTuning(Kp_OMV_SOPDT, T1_OMV_SOPDT, T2_OMV_SOPDT, theta_OMV_SOPDT, gamma, model="SOPDT")
  
def stop_experiment_button_clicked(b):
    global should_continue
    should_continue = False  # Set the flag to False to stop the loop

def Update_alpha(alphaP):
    global alphaBuffer
    alphaBuffer = alphaP
    
def Update_Manual(manual_time, MVManual, ActiveManual):
    global ManPathBuffer
    global MVmanPathBuffer
    global ManFFBuffer
     
    if ActiveManual:
        ManPathBuffer = {0: False, manual_time[0]: True, manual_time[1]: False}
        MVmanPathBuffer = {0: 0, manual_time[0]: MVManual, manual_time[1]: 0}
        ManFFBuffer = True; 
    else :
        ManFFBuffer = False
        ManPathBuffer = {0: False}
        MVmanPathBuffer = {0: 0}


def Update_Perturbation(perturbation, time_perturbation):
    global DVPathBuffer
    DVPathBuffer = {0: DV0, time_perturbation[0]: perturbation[0], time_perturbation[1]: perturbation[1]}
    
def Update_SetPoint(setpoint, time_setpoint):
    global SPPathBuffer
    SPPathBuffer = {0: PV0, time_setpoint[0]: setpoint[0], time_setpoint[1]: setpoint[1]}

def Update_FF(ActiveFF):
    global FFBuffer
    FFBuffer = ActiveFF

def Update_TimeStep(_timeStep):
    global TimeStep
    TimeStep = _timeStep
    
def run_experiment_threaded():
    # Wrapper function to run the experiment in a separate thread
    RunExp(Exp=True)

def run_experiment_button_clicked(b):
    global should_continue
    should_continue = True  # Ensure the flag is reset to True when starting
    
    if not fig.data:
        initialize()
        reinitialize()
    
    # Start the experiment in a separate thread to keep the UI responsive
    experiment_thread = threading.Thread(target=run_experiment_threaded)
    experiment_thread.start()


def stop_experiment_button_clicked(b):
    global should_continue
    should_continue = False  # Set the flag to False to stop the loop
    
def clear_graph_button_clicked(b):
    # Clear the graph by resetting its data
    fig.data = []
    
run_exp_button = widgets.Button(
    description='Run Experiment',
    button_style='info', 
    tooltip='Click to run the experiment',
    icon='play' 
)

stop_exp_button = widgets.Button(
    description='Stop Experiment',
    button_style='danger',
    tooltip='Click to stop the experiment',
    icon='stop'
)

clear_graph_button = widgets.Button(
    description='Clear Graph',
    button_style='warning',  # Adjust the button style as needed
    tooltip='Click to clear the graph',
    icon='eraser'  # FontAwesome icon name
)

run_exp_button.on_click(run_experiment_button_clicked)
stop_exp_button.on_click(stop_experiment_button_clicked)
clear_graph_button.on_click(clear_graph_button_clicked)


slider_style = {'description_width': 'initial'}
manual_time_slider = IntRangeSlider(min=0, max=1999, step=1, value=[50, 100], description="Manual Time", layout={'width': '500px'}, slider_style=slider_style)
MVManual_slider = IntSlider(min=0, max=100, step=1, value=50, description="MV Manual Value", layout={'width': '500px'}, slider_style=slider_style)
ActiveManual_checkbox = Checkbox(value=False, description="Active Manual", layout={'width': '500px'}, slider_style=slider_style)
PerturbationSlider = IntRangeSlider(min=0, max=100, step=1, value=[50, 100], description="Perturbation Value", layout={'width': '500px'}, slider_style=slider_style)
time_perturbation = IntRangeSlider(min=0, max=1999, step=10, value=[50, 100], description="Perturbation Time", layout={'width': '500px'}, slider_style=slider_style)
SetPointSlider = IntRangeSlider(min=0, max=100, step=1, value=[50, 100], description="SetPoint Value", layout={'width': '500px'}, slider_style=slider_style)
time_setpoint = IntRangeSlider(min=0, max=1999, step=10, value=[50, 100], description="SetPoint Time", layout={'width': '500px'}, slider_style=slider_style)
time_slider = IntSlider(min=0, max=10, step=1, value=1, description="Simulation Time (per loop)", layout={'width': '500px'}, orientation='vertical', slider_style=slider_style)

# Create interactive widget
GammaSlider = interactive(Update_gamma, gamma=(0.2, 0.9, 0.02), description='Gamma')
AlphaSlider = interactive(Update_alpha, alphaP=(0, 0.9, 0.01), description='Alpha')
Manual = interactive(Update_Manual, manual_time=manual_time_slider, MVManual=MVManual_slider, ActiveManual=ActiveManual_checkbox)
FFCheckBox = interactive(Update_FF, ActiveFF=False, description='FeedForward')
Perturbation = interactive(Update_Perturbation, perturbation=PerturbationSlider, time_perturbation=time_perturbation, description='Perturbation')
SetPoint = interactive(Update_SetPoint, setpoint=SetPointSlider, time_setpoint=time_setpoint, description='SetPoint')
timeStep = interactive(Update_TimeStep, _timeStep=time_slider, description='Time Step')

# Display figure and widget
vb = VBox([fig, Manual, GammaSlider, AlphaSlider, FFCheckBox, Perturbation, SetPoint, run_exp_button, stop_exp_button, clear_graph_button, timeStep])
vb.layout.align_items = 'center'
vb  

    'data': [{'name': 'SP',
              'type': 'scatter',
              'uid'…

Exception in thread Thread-27 (run_experiment_threaded):
Traceback (most recent call last):
  File "c:\Users\xavier\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\xavier\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\xavier\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\xavier\AppData\Local\Temp\ipykernel_7116\4123722788.py", line 139, in run_experiment_threaded
  File "C:\Users\xavier\AppData\Local\Temp\ipykernel_7116\4123722788.py", line 72, in RunExp
AttributeError: 'FigureWidget' object has no attribute 'extend_traces'. Did you mean: 'append_trace'?
Exception in thread Thread-28 (run_experiment_threaded):
Traceback (most recent call last):
  File "c:\Users\xavier\AppData\Local\Programs\Python\Python312\Lib\thr